In [1]:
!pip install tensorflow keras numpy pillow


In [6]:
!unzip -q /content/drive/MyDrive/training_dataset.zip -d /content/drive/MyDrive/train_dataset

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
image_size = (150, 150)  # Resize images to 150x150 pixels
batch_size = 32

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train_dataset/training_dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 2527 images belonging to 6 classes.
Found 402 images belonging to 6 classes.


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 31s 347ms/step - accuracy: 0.2394 - loss: 1.9464 - val_accuracy: 0.3021 - val_loss: 1.6576
Epoch 2/10
 1/78 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.4062 - loss: 1.5015

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4062 - loss: 1.5015 - val_accuracy: 0.1667 - val_loss: 1.7049
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 36s 308ms/step - accuracy: 0.4791 - loss: 1.3187 - val_accuracy: 0.6432 - val_loss: 0.9514
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5000 - loss: 1.2797 - val_accuracy: 0.8333 - val_loss: 0.7754
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 39s 302ms/step - accuracy: 0.5609 - loss: 1.1656 - val_accuracy: 0.6927 - val_loss: 0.8260
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5938 - loss: 0.9665 - val_accuracy: 0.7222 - val_loss: 0.9043
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 27s 319ms/step - accuracy: 0.5807 - loss: 1.0911 - val_accuracy: 0.7214 - val_loss: 0.7746
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - accuracy: 0.6875 - loss: 0.8239 - val_accuracy: 0.7222 - val_loss: 0.7951
Epoch 9/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 26s 305ms/step - accuracy: 0.6308 - loss: 0.9840 - val_accuracy: 0.7370 - val

In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess image
img = image.load_img('/content/drive/MyDrive/dataset/plastic/plastic14.jpg', target_size=image_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Create batch axis

# Predict
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
class_labels = list(train_generator.class_indices.keys())
print(f'Predicted class: {class_labels[predicted_class[0]]}')
print(f'Probabilities: {predictions}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted class: plastic
Probabilities: [[0. 0. 0. 0. 1. 0.]]


In [22]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [29]:
import gradio as gr
from tensorflow.keras.preprocessing import image
import numpy as np


# Define image size used during model training
image_size = (150, 150)  # Adjust if necessary

def predict_image(img):

    # Load and preprocess image
    img = img.resize(image_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    img_array = img_array / 255.0  # Normalize pixel values

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    class_labels = list(train_generator.class_indices.keys())

    return class_labels[predicted_class[0]]
    # return predictions[0].tolist()


In [ ]:
# Create Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Updated syntax for input
    outputs=gr.Textbox(label="Predicted Class")   # Updated syntax for outputs
)
# Launch the interface
interface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4d3839c16507d4b3cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
